In [2]:
import glob
from os.path import join as oj

import numpy as np
import os
import pandas as pd
from tqdm import tqdm
from typing import Dict
import functools

import rulevetting
from rulevetting.api import validation, util as api_util
from rulevetting.projects.csi_pecarn.dataset1 import Dataset
from rulevetting import DATA_PATH


In [3]:
raw_data_path = oj(rulevetting.DATA_PATH, Dataset().get_dataset_id(), 'raw')
fnames = sorted(glob.glob(f'{raw_data_path}/*'))
dfs = [pd.read_csv(fname) for fname in fnames]
dfs[0].loc[:, 'ControlType'] = (dfs[0]['ControlType'] == 'case').astype(int)
# fname=glob.glob(f'{raw_data_path}/clinicalpresentationsite.csv')[0]
# dfs = pd.read_csv(fname)
# dfs = dfs.replace(['Y', 'YES', 'A'], 1)       
# dfs = dfs.replace(['N', 'NO'], 0)
# dfs = dfs.replace(['ND', 'NA'], float("NaN"))
# dfs=dfs.fillna(dfs.median()) 

# Correlation of features and in groups and association with outcome


## Group1: Consciousness

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
dfs[3].columns

Index(['SITE', 'CaseID', 'ControlType', 'StudySubjectID', 'ArrivalDate',
       'ArrivalTime', 'ArrivalTimeND', 'ModeArrival', 'ReceivedInTransfer',
       'DxCspineInjury',
       ...
       'IntervForCervicalStabOthertxt', 'LongTermRehab', 'TrfToLongTermRehab',
       'OutcomeStudySite', 'OutcomeStudySiteNeuro', 'OutcomeStudySiteMobility',
       'OutcomeStudySiteMobility1', 'OutcomeStudySiteMobility2',
       'OutcomeStudySiteBowel', 'OutcomeStudySiteUrine'],
      dtype='object', length=136)

In [36]:
feat_conscious = ['HxLOC', 'TotalGCSManual', 'TotalGCS', 'AVPUDetails','AlteredMentalStatus', 'LOC','ControlType_x']
dfs_conscious=dfs[0].merge(dfs[3],how='left', on=['SITE', 'CaseID', 'StudySubjectID'])
dfs_conscious=dfs_conscious[feat_conscious]
# dfs_conscious.loc[:, 'ControlType_x'] = (dfs_conscious['ControlType_x'] == 'case').astype(int)

dfs_conscious = dfs_conscious.replace(['Y', 'YES', 'A'], 1)       
dfs_conscious = dfs_conscious.replace(['N', 'NO'], 0)
dfs_conscious = dfs_conscious.replace(['15'], 15)
dfs_conscious = dfs_conscious.replace(['10'], 10)
dfs_conscious = dfs_conscious.replace(['14'], 14)
dfs_conscious = dfs_conscious.replace(['6'], 6)
dfs_conscious = dfs_conscious.replace(['8'], 8)
dfs_conscious = dfs_conscious.replace(['12'], 12)
dfs_conscious = dfs_conscious.replace(['5'], 5)
dfs_conscious = dfs_conscious.replace(['13'], 13)
dfs_conscious = dfs_conscious.replace(['11'],11)
dfs_conscious = dfs_conscious.replace(['9'], 9)
dfs_conscious = dfs_conscious.replace(['7','7T'], 7)
dfs_conscious = dfs_conscious.replace(['4'], 4)
dfs_conscious = dfs_conscious.replace(['ND', 'NA', '3'], float("NaN"))
dfs_conscious = dfs_conscious.replace(['U','V','N', 'P'], 0)
# dfs_conscious=dfs_conscious.fillna(dfs_conscious.median()) 
# print(dfs_conscious['TotalGCS'])
# print(pd.unique(dfs_conscious['TotalGCS']))
dfs_conscious = dfs_conscious.rename(columns={'ControlType_x': 'outcome'})
dfs_conscious.corr(method='pearson').style.background_gradient(cmap="Blues")


,HxLOC,TotalGCSManual,TotalGCS,AVPUDetails,AlteredMentalStatus,LOC,outcome
HxLOC,1.000000,-0.409745,-0.300756,-0.322035,0.368531,0.945069,0.042620
TotalGCSManual,-0.409745,1.000000,nan,0.697871,-0.710236,-0.358280,-0.131194
TotalGCS,-0.300756,nan,1.000000,0.678785,-0.726675,-0.280861,-0.105658
AVPUDetails,-0.322035,0.697871,0.678785,1.000000,-0.797965,-0.293623,-0.089832
AlteredMentalStatus,0.368531,-0.710236,-0.726675,-0.797965,1.000000,0.352847,0.117775
LOC,0.945069,-0.358280,-0.280861,-0.293623,0.352847,1.000000,0.027341
outcome,0.042620,-0.131194,-0.105658,-0.089832,0.117775,0.027341,1.000000


## Group2: Complaint of pain in neck and age


In [35]:
# feat_pain = ['PtCompPainHead', 'PtCompPainFace', 'PtCompPainNeck', 'PtCompPainNeckMove', 'PtCompPainChest', 'PtCompPainBack', 'PtCompPainFlank', 'PtCompPainAbd', 'PtCompPainPelvis', 'PtCompPainExt']
# demog_df = dfs[4]
clean_key_col_names = lambda df: df.rename(columns={'site': 'SITE',
                                                            'caseid': 'CaseID',
                                                            'studysubjectid': 'StudySubjectID'})
demog_df = clean_key_col_names(dfs[4])

agegroup_df = pd.get_dummies(pd.cut(demog_df['AgeInYears'], bins=[0, 2, 6, 12, 16],
                                            labels=['infant', 'preschool', 'school_age', 'adolescents'],
                                            include_lowest=True), prefix='age')
agegroup_df=pd.concat([demog_df[['SITE', 'CaseID', 'StudySubjectID']], agegroup_df], axis=1)

feat_pain = ['PtCompPainNeck', 'PtCompPainNeckMove', 'age_infant', 'age_preschool',  'age_school_age',  'age_adolescents','PainNeck','ControlType_x']
dfs_pain=dfs[0].merge(dfs[3],how='left', on=['SITE', 'CaseID', 'StudySubjectID'])
dfs_pain=dfs_pain.merge(agegroup_df,how='left', on=['SITE', 'CaseID', 'StudySubjectID'])
dfs_pain=dfs_pain[feat_pain]
dfs_pain = dfs_pain.replace(['Y', 'YES', 'A'], 1)       
dfs_pain = dfs_pain.replace(['N', 'NO'], 0)
dfs_pain = dfs_pain.replace(['ND', 'NA'], float("NaN"))
dfs_pain = dfs_pain.rename(columns={'ControlType_x': 'outcome'})
# dfs_pain=dfs_pain.fillna(dfs_pain.median()) 
# print(pd.unique(dfs_pain['PtCompPainNeckMove']))
dfs_pain.corr(method='pearson').style.background_gradient(cmap="Blues")

,PtCompPainNeck,PtCompPainNeckMove,age_infant,age_preschool,age_school_age,age_adolescents,PainNeck,outcome
PtCompPainNeck,1.000000,nan,-0.189416,-0.107573,0.044036,0.140669,1.000000,0.123322
PtCompPainNeckMove,nan,1.000000,nan,0.111897,-0.102825,0.006222,nan,0.114576
age_infant,-0.189416,nan,1.000000,-0.131898,-0.167260,-0.242185,-0.202944,-0.028564
age_preschool,-0.107573,0.111897,-0.131898,1.000000,-0.311781,-0.451444,-0.086399,-0.005268
age_school_age,0.044036,-0.102825,-0.167260,-0.311781,1.000000,-0.572475,0.043904,-0.025394
age_adolescents,0.140669,0.006222,-0.242185,-0.451444,-0.572475,1.000000,0.130484,0.041454
PainNeck,1.000000,nan,-0.202944,-0.086399,0.043904,0.130484,1.000000,0.123750
outcome,0.123322,0.114576,-0.028564,-0.005268,-0.025394,0.041454,0.123750,1.000000


## Group3: Tenderness in neck


In [34]:
# feat_tender = ['PtTenderHead', 'PtTenderFace', 'PtTenderNeck', 'PtTenderNeckLevel', 'PtTenderNeckLevelC1', 'PtTenderNeckLevelC2', 'PtTenderNeckLevelC3', 'PtTenderNeckLevelC4', 'PtTenderNeckLevelC5', 'PtTenderNeckLevelC6', 'PtTenderNeckLevelC7', 'PtTenderNeckAnt', 'PtTenderNeckPos', 'PtTenderNeckLat', 'PtTenderNeckMid', 'PtTenderNeckOther', 'PtTenderChest', 'PtTenderBack', 'PtTenderFlank', 'PtTenderAbd', 'PtTenderPelvis', 'PtTenderExt']
feat_tender =['PtTenderNeck', 'PtTenderNeckLevel', 'PtTenderNeckLevelC1', 'PtTenderNeckLevelC2', 'PtTenderNeckLevelC3', 'PtTenderNeckLevelC4', 'PtTenderNeckLevelC5', 'PtTenderNeckLevelC6', 'PtTenderNeckLevelC7', 'PtTenderNeckAnt', 'PtTenderNeckPos', 'PtTenderNeckLat', 'PtTenderNeckMid', 'PtTenderNeckOther','PosMidNeckTenderness', 'TenderNeck','ControlType_x'] 

dfs_tender=dfs[0].merge(dfs[3],how='left', on=['SITE', 'CaseID', 'StudySubjectID'])
dfs_tender=dfs_tender[feat_tender]
dfs_tender = dfs_tender.replace(['Y', 'YES', 'A'], 1)       
dfs_tender = dfs_tender.replace(['N', 'NO'], 0)
dfs_tender = dfs_tender.replace(['ND', 'NA'], float("NaN"))
# dfs_tender=dfs_tender.fillna(dfs_tender.median()) 
# print(dfs_tender)
dfs_tender = dfs_tender.rename(columns={'ControlType_x': 'outcome'})
hide_column_neck=["PtTenderNeck", "PtTenderNeckLevel","PtTenderNeckLevelC1","PtTenderNeckLevelC2", "PtTenderNeckLevelC3","PtTenderNeckLevelC4","PtTenderNeckLevelC5","PtTenderNeckLevelC6","PtTenderNeckLevelC7","PtTenderNeckAnt","PtTenderNeckPos","PtTenderNeckLat","PtTenderNeckMid","PtTenderNeckOther","PosMidNeckTenderness","TenderNeck"]
dfs_tender_corr=dfs_tender.corr(method='pearson').style.background_gradient(cmap="Blues").hide_columns(hide_column_neck)
dfs_tender_corr

,outcome
PtTenderNeck,0.040167
PtTenderNeckLevel,0.029306
PtTenderNeckLevelC1,0.030859
PtTenderNeckLevelC2,0.030202
PtTenderNeckLevelC3,0.008022
PtTenderNeckLevelC4,0.013563
PtTenderNeckLevelC5,0.022655
PtTenderNeckLevelC6,0.045919
PtTenderNeckLevelC7,0.065611
PtTenderNeckAnt,-0.018944


## Group4: Focal neurological deficits

In [33]:
feat_focal= ['PtParesthesias', 'PtSensoryLoss', 'PtExtremityWeakness', 'OtherNeuroDeficitDescCat','FocalNeuroFindings','ControlType_x']
dfs_focal=dfs[0].merge(dfs[3],how='left', on=['SITE', 'CaseID', 'StudySubjectID'])
dfs_focal=dfs_focal[feat_focal]
dfs_focal = dfs_focal.replace(['Y', 'YES', 'A'], 1)       
dfs_focal = dfs_focal.replace(['N', 'NO'], 0)
dfs_focal = dfs_focal.replace(['3'], 3)
dfs_focal = dfs_focal.replace(['ND', 'NA'], float("NaN"))
dfs_focal = dfs_focal.rename(columns={'ControlType_x': 'outcome'})
# dfs_focal=dfs_focal.fillna(dfs_focal.median()) 
# print(pd.unique(dfs_focal['PtExtremityWeakness']))
dfs_focal.corr(method='pearson').style.background_gradient(cmap="Blues")

,PtParesthesias,PtSensoryLoss,PtExtremityWeakness,OtherNeuroDeficitDescCat,FocalNeuroFindings,outcome
PtParesthesias,1.000000,0.935326,0.906438,0.258781,0.256178,0.149634
PtSensoryLoss,0.935326,1.000000,0.929030,0.284431,0.224048,0.163905
PtExtremityWeakness,0.906438,0.929030,1.000000,0.314892,0.251376,0.188315
OtherNeuroDeficitDescCat,0.258781,0.284431,0.314892,1.000000,0.844679,0.477683
FocalNeuroFindings,0.256178,0.224048,0.251376,0.844679,1.000000,0.303406
outcome,0.149634,0.163905,0.188315,0.477683,0.303406,1.000000


## Group 5: Other parts of the body

In [32]:
feat_otherpain = ['PtCompPainHead', 'PtCompPainFace', 'PtCompPainExt', 'PtTenderHead', 'PtTenderFace', 'PtTenderExt','SubInj_Head', 'SubInj_Face', 'SubInj_Ext', 'SubInj_TorsoTrunk','ControlType_x']
dfs_otherpain=dfs[0].merge(dfs[3],how='left', on=['SITE', 'CaseID', 'StudySubjectID'])
dfs_otherpain=dfs_otherpain[feat_otherpain]
dfs_otherpain = dfs_otherpain.replace(['Y', 'YES', 'A'], 1)       
dfs_otherpain = dfs_otherpain.replace(['N', 'NO'], 0)
dfs_otherpain = dfs_otherpain.replace(['ND', 'NA'], float("NaN"))
dfs_otherpain = dfs_otherpain.rename(columns={'ControlType_x': 'outcome'})
# dfs_focal=dfs_focal.fillna(dfs_focal.median()) 
# print(dfs_focal)
dfs_otherpain.corr(method='pearson').style.background_gradient(cmap="Blues")

,PtCompPainHead,PtCompPainFace,PtCompPainExt,PtTenderHead,PtTenderFace,PtTenderExt,SubInj_Head,SubInj_Face,SubInj_Ext,SubInj_TorsoTrunk,outcome
PtCompPainHead,1.000000,0.023547,0.037793,0.262369,0.016954,0.010375,-0.044970,-0.075680,-0.106077,-0.117747,-0.093533
PtCompPainFace,0.023547,1.000000,0.075118,0.011725,0.374723,0.056732,-0.048357,0.201582,-0.014918,-0.023898,-0.075342
PtCompPainExt,0.037793,0.075118,1.000000,0.010720,0.053050,0.561550,-0.133816,-0.059482,0.249719,-0.017379,-0.088681
PtTenderHead,0.262369,0.011725,0.010720,1.000000,0.099348,0.022536,0.019453,-0.025788,-0.055554,-0.044621,-0.026963
PtTenderFace,0.016954,0.374723,0.053050,0.099348,1.000000,0.081796,-0.024345,0.198127,-0.018845,-0.012344,-0.050347
PtTenderExt,0.010375,0.056732,0.561550,0.022536,0.081796,1.000000,-0.084606,-0.018728,0.199783,-0.000556,-0.091866
SubInj_Head,-0.044970,-0.048357,-0.133816,0.019453,-0.024345,-0.084606,1.000000,0.213585,0.117020,0.138892,0.061941
SubInj_Face,-0.075680,0.201582,-0.059482,-0.025788,0.198127,-0.018728,0.213585,1.000000,0.061393,0.093213,-0.013333
SubInj_Ext,-0.106077,-0.014918,0.249719,-0.055554,-0.018845,0.199783,0.117020,0.061393,1.000000,0.164603,-0.000852
SubInj_TorsoTrunk,-0.117747,-0.023898,-0.017379,-0.044621,-0.012344,-0.000556,0.138892,0.093213,0.164603,1.000000,0.090464


## Group6: Injury mechanism


In [31]:
feat_injury= ['InjuryPrimaryMechanism', 'HeadFirst', 'HeadFirstRegion','HighriskDiving', 'HighriskFall', 'HighriskHanging', 'HighriskHitByCar', 'HighriskMVC', 'HighriskOtherMV', 'AxialLoadAnyDoc', 'axialloadtop', 'Clotheslining','ControlType_x']
dfs_injury=dfs[0].merge(dfs[6],how='left', on=['SITE', 'CaseID', 'StudySubjectID'])
dfs_injury=dfs_injury[feat_injury]
dfs_injury = dfs_injury.replace(['Y', 'YES', 'A'], 1)       
dfs_injury = dfs_injury.replace(['N', 'NO'], 0)
dfs_injury = dfs_injury.replace(['ND', 'NA'], float("NaN"))
dfs_injury=dfs_injury.rename(columns={'ControlType_x': 'outcome'})
# dfs_focal=dfs_focal.fillna(dfs_focal.median()) 
# print(pd.unique(dfs_injury['HeadFirst']))
hide_column_injury=["HeadFirst","HighriskDiving","HighriskFall","HighriskHanging","HighriskHitByCar","HighriskMVC","HighriskOtherMV","AxialLoadAnyDoc","axialloadtop","Clotheslining"]
dfs_injury.corr(method='pearson').style.background_gradient(cmap="Blues").hide_columns(hide_column_injury)

,outcome
HeadFirst,0.101228
HighriskDiving,0.202099
HighriskFall,-0.040855
HighriskHanging,-0.017156
HighriskHitByCar,-0.062772
HighriskMVC,0.066794
HighriskOtherMV,0.005705
AxialLoadAnyDoc,0.071673
axialloadtop,0.087797
Clotheslining,0.060762
